In [1]:
.include header

In [2]:
%%file header/monomial.h
#pragma once

#include "tensor_product.h"
#include "eliastocco_namespace_polynomial.h"
#include "basis.h"
#include "evaluate.h"


namespace eliastocco {

    namespace polynomial {
    
    
        template < class type , t_dim dimension >
            basis < type , dimension > monomial_basis (const t_deg<dimension> N ){
                
                //costruttore
                return basis < type , dimension > ( make_monomials<type,dimension>(N).value );

            };
        
        
        template < class type >
            class make_monomials<type,1> {
                
                public :
                    make_monomials<type,1> ( const t_deg<1> N ){                        
                        
                         value = std :: make_shared < t_basis_function < type , 1 > >();
                     
                         for (auto i=0;i<=N;++i){
                    
                            (*value).insert(std :: make_pair(i,[=](const t_var<type,1> x){ return pow(x,i); }));
                         }
                        
                    }
                
                public :
                    t_ptr < type , 1 > value;
                
            };
        
    
        template < class type , t_dim dimension >
            class make_monomials {
                
                public :
                    make_monomials<type,dimension> ( const t_deg<dimension> N ){
                        
                        t_var<t_degree,1> N_0;
                        t_var<t_degree,dimension-1> N_1;
                                
                        tensor_product::split_array<t_degree,1,dimension-1> X (N);
                        std :: tie (N_0,N_1) =  X.value;
            
                        auto basis_1  = *(make_monomials<type,1> (N_0).value);
                        
                        auto basis_N_inf = *(make_monomials <type,dimension-1>(N_1).value);
           
                        auto T = tensor_product :: tensor_product<type,1,dimension-1>( basis_1 , basis_N_inf ).value;
                        
                        value = std :: make_shared<decltype(T)>(T);
                        
                    }
                
                public :
                    t_ptr < type , dimension > value;
                        
                        
            };  
      
    
    }
    
}

Overwriting header/monomial.h


In [3]:
#include "monomial.h"

In [4]:
using namespace eliastocco :: polynomial;

In [5]:
auto A = *(make_monomials<double> (3).value)

In [6]:
A.size()

4

In [7]:
auto B = *(make_monomials<double,1> (3).value)

In [8]:
auto C = *(make_monomials<double,2> ({3,3}).value)

In [9]:
auto T = tensor_product::tensor_product<double,1,1>(A,B).value;

In [10]:
auto T3 = tensor_product::tensor_product<double,1,2>(A,T).value;

In [11]:
auto t3 = T3 [{0,0,0}]

In [12]:
t3({2,3,4})

1.0000000

In [13]:
//auto M3 = *(make_monomials<double,3> ({2,3,4}).value)

In [14]:
auto base_polinomiale_1D = monomial_basis<double,1>(3)

In [15]:
base_polinomiale_1D.size()

4

In [16]:
auto base_polinomiale_2D = monomial_basis<double,2>({3,4})

In [17]:
base_polinomiale_2D.size()

20

In [19]:
auto p = base_polinomiale_2D[{0,0}]

In [20]:
p({2,2})

1.0000000


/*   
    template < class type > 
        class monomial_basis : public basis < type > {
            
            #define thisclass monomial_basis<type>      
            
            private :
                thisclass () {};
            
            //public :
            //    thisclass (const t_basis_function<type> user_monomials ) : monomials (user_monomials) {}
            
            public :
                thisclass (const unsigned int N) 
                    : thisclass ( N , eliastocco :: default_polynomial_evaluation < type > ) {};
              
            public :
                thisclass (const unsigned int N, const t_eval<type> user_evaluate) 
                    : basis<type> (make_monomials <type> (N)){
                    
                    assert ( N > 0 && "polynomial degree must be greater than 0");
                    // = monomials = ;                    
                    _evaluate = user_evaluate;
                    
                };         
            
            //virtual specialization
            public :
                type evaluate (const t_coeff_array<type> coeff, const type x) override { 
                    return _evaluate(coeff,(*this),x); 
                };
            
           
            
            private :
                //t_ptr<type> monomials;
                t_eval<type> _evaluate;
            
            #undef thisclass      
            
        };
        
        */
    

In [ ]:
//%%file header/monomial.h
#pragma once

#include "eliastocco_namespace.h"
#include "basis.h"
#include "tensor_product.h"

namespace eliastocco {

    namespace polynomial {
    
        
        template < class type >
            basis < type , 1 > monomial_basis (const t_deg<1> N ){
                
                //costruttore
                return basis < type , 1 > ( make_monomials<type>(N) );

            };
        
        /*template < class type >
            basis < type , 2 > monomial_basis (const t_deg<2> N ){
                
                //costruttore
                return basis < type , 2 > ( make_monomials<type>(N) );

            };*/
        
        template < class type , t_dim dimension >
            basis < type , dimension > monomial_basis (const t_deg<dimension> N ){
                
                //costruttore
                return basis < type , dimension > ( make_monomials<type,dimension>(N) );

            };
        
        
        
        
        
        //template function overloading
        /* template < class type >
             t_ptr < type , 1 > make_monomials (const std :: array < t_degree,1> N ){ 
             
                 return make_monomials<type>(N[0]);
         
         };*/
    
         //template function overloading
         template < class type >
             t_ptr < type , 1 > make_monomials (const t_deg<1> N){

                 t_ptr < type , 1 > output = std :: make_shared < t_basis_function < type , 1 > >();
                     
                 for (auto i=0;i<=N;++i){
                    //I[0]=i;
                    (*output).insert(std :: make_pair(i,[=](const t_var<type,1> x){ return pow(x,i); }));
                 }
                    return output;
                };
        
         //template function implementation
        template < class type >
            t_ptr < type , 2 > make_monomials (const t_deg<2> N){
            
                //static_assert ( dimension > 2 , "dimension is not greater than 2");
                //creo la base polinomiale di grado 1
                auto basis_1  = *make_monomials<type> (N[0]+1);
            
                auto basis_2  = *make_monomials<type> (N[1]+1);
            
                auto T = tensor_product :: tensor_product<type>( basis_1 , basis_2 );
            
                return std :: make_shared<decltype(T)>(T);

            
            };
        
        //template function implementation
        template < class type , t_dim dimension >
            t_ptr < type , dimension > make_monomials (const t_deg<dimension> N){
            
                //static_assert ( dimension > 2 , "dimension is not greater than 2");
                //creo la base polinomiale di grado 1
            
                t_var<t_degree,1> N_0;
                t_var<t_degree,dimension-1> N_1;
                                
                tensor_product::split_array<t_degree,1,dimension-1> X (N);
                std :: tie (N_0,N_1) =  X.value;
            
                auto basis_1  = *make_monomials<type> (N_0+1);
            
                
                //t_ptr < type , dimension-1> basis_N_inf  ;
            
                //creo un array con i rimanenti gradi polinomiali
                //t_deg <dimension-1> N_inf ;
                //for ( auto i = 0; i < (dimension-1);++i){ N_inf[i]=N[i+1]; }
                    
                //creo la base polinomiale di grado inferiore
                auto basis_N_inf = *make_monomials < type >(N_1);
           
                auto T = tensor_product :: tensor_product<type,1,dimension-1>( basis_1 , basis_N_inf );
            
                return std :: make_shared<decltype(T)>(T);

            
            };
    
    }
    
}